In [1]:
import benford
import numpy as np
import pandas as pd
import os

In [2]:
os.system('pwd')
datafile = 'tests/census_2009b.csv'
census_df = pd.read_csv(datafile, sep = '\t')

/home/matt/l7/D-fraudr


In [3]:
census_df


,State,Town,7_2009,3,4,8.40188
0,Alabama,Abbeville,2930,3.0,10.0,3.94383
1,Alabama,Adamsville,4782,3.0,11.0,7.83099
2,Alabama,Addison,709,3.0,8.0,7.98440
3,Alabama,Akron,433,3.0,6.0,9.11647
4,Alabama,Alabaster,29861,3.0,10.0,1.97551
...,...,...,...,...,...,...
19504,Wyoming,Wamsutter,310,3.0,10.0,1.75974
19505,Wyoming,Wheatland,3236,3.0,10.0,1.35188
19506,Wyoming,Worland,5054,3.0,8.0,1.03296
19507,Wyoming,Wright,1550,3.0,7.0,9.55124


In [4]:
pop_data = census_df['7_2009'][census_df['7_2009'] > 0]
pop_data

0         2930
1         4782
2          709
3          433
4        29861
         ...  
19504      310
19505     3236
19506     5054
19507     1550
19508      164
Name: 7_2009, Length: 19507, dtype: int64

In [5]:
#str(pop_data.dtype) == 'int64'
'int' in str(pop_data.dtype)

True

In [6]:
type(pop_data)

pandas.core.series.Series

In [7]:
import hashlib

In [8]:
import psycopg2
import psycopg2.extras
def get_db():
    conn = psycopg2.connect(
        host = "localhost",
        database = "dfraudr",
        user = "dfraud_user",
        password = "dfraud_pass")
    conn.set_session(autocommit=True)
    cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

    return cur

In [9]:
cur = get_db()

In [10]:
cur.execute('SELECT id, filename, description, md5sum FROM df_datafiles')
files = cur.fetchall()
files

[[1, 'census_2009b.csv', 'Untitled', None]]

In [11]:
print(type(files))
print(type(files[0]))
files[0]['id']

<class 'list'>
<class 'psycopg2.extras.DictRow'>


1

In [12]:
(type(files) == 'NoneType')

False

In [13]:
not files

False

In [14]:
cur

<cursor object at 0x7f7e4eb93440; closed: 0>

In [15]:
cols = census_df.columns.array
cols

<PandasArray>
['State', 'Town', '7_2009', '3', '4', '8.40188']
Length: 6, dtype: object

In [16]:
str((census_df[cols[5]]).dtype)

'float64'

In [17]:
floater = census_df['8.40188']
floater

0        3.94383
1        7.83099
2        7.98440
3        9.11647
4        1.97551
          ...   
19504    1.75974
19505    1.35188
19506    1.03296
19507    9.55124
19508    8.66964
Name: 8.40188, Length: 19509, dtype: float64

In [18]:
floater.min()

0.000468777

In [19]:
floater.max()

9.99994

In [20]:
for column in census_df.columns:
    print(column.dtype)

AttributeError: 'str' object has no attribute 'dtype'

In [21]:
types = census_df.dtypes
types

State       object
Town        object
7_2009       int64
3          float64
4          float64
8.40188    float64
dtype: object

In [22]:
str(types[0])

'object'

In [23]:
info = census_df.describe()

In [24]:
info

,7_2009,3,4,8.40188
count,1.950900e+04,19509.000000,19507.000000,19503.000000
mean,9.793011e+03,3.004910,9.549200,4.964280
std,8.202300e+04,0.352347,2.890984,2.886162
min,0.000000e+00,0.303240,0.650635,0.000469
25%,3.650000e+02,3.000000,7.000000,2.469750
50%,1.115000e+03,3.000000,9.000000,4.940950
75%,4.498000e+03,3.000000,11.000000,7.448870
max,8.391881e+06,35.000000,32.000000,9.999940


In [25]:
bad_df= pd.read_csv(datafile, sep=None)

/tmp/ipykernel_2600/2628795222.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  bad_df= pd.read_csv(datafile, sep=None)


In [26]:
bad_info = bad_df.describe()
bad_info

,7_2009,3,4,8.40188
count,1.950900e+04,19509.000000,19507.000000,19503.000000
mean,9.793011e+03,3.004910,9.549200,4.964280
std,8.202300e+04,0.352347,2.890984,2.886162
min,0.000000e+00,0.303240,0.650635,0.000469
25%,3.650000e+02,3.000000,7.000000,2.469750
50%,1.115000e+03,3.000000,9.000000,4.940950
75%,4.498000e+03,3.000000,11.000000,7.448870
max,8.391881e+06,35.000000,32.000000,9.999940


In [27]:
stats = bad_info.columns.array

In [28]:
numbercols = [bad_df[x] for x in stats]

In [29]:
type(numbercols[0])

pandas.core.series.Series

In [30]:
id = 8
cursor=get_db()
cursor.execute('SELECT * FROM df_datafiles WHERE id=%s', (id,))

In [31]:
file = cursor.fetchone()

In [32]:
file

In [33]:
bad_info.index

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')

In [34]:
for column in bad_info.index:
    print(column)

count
mean
std
min
25%
50%
75%
max


In [35]:
bad_info['7_2009']

count    1.950900e+04
mean     9.793011e+03
std      8.202300e+04
min      0.000000e+00
25%      3.650000e+02
50%      1.115000e+03
75%      4.498000e+03
max      8.391881e+06
Name: 7_2009, dtype: float64

In [36]:
for column in numbercols:
    print(column)

0         2930
1         4782
2          709
3          433
4        29861
         ...  
19504      310
19505     3236
19506     5054
19507     1550
19508      164
Name: 7_2009, Length: 19509, dtype: int64
0        3.0
1        3.0
2        3.0
3        3.0
4        3.0
        ... 
19504    3.0
19505    3.0
19506    3.0
19507    3.0
19508    3.0
Name: 3, Length: 19509, dtype: float64
0        10.0
1        11.0
2         8.0
3         6.0
4        10.0
         ... 
19504    10.0
19505    10.0
19506     8.0
19507     7.0
19508     6.0
Name: 4, Length: 19509, dtype: float64
0        3.94383
1        7.83099
2        7.98440
3        9.11647
4        1.97551
          ...   
19504    1.75974
19505    1.35188
19506    1.03296
19507    9.55124
19508    8.66964
Name: 8.40188, Length: 19509, dtype: float64


In [37]:
numbercols.index

<function list.index(value, start=0, stop=9223372036854775807, /)>

In [38]:
bad_df.columns

Index(['State', 'Town', '7_2009', '3', '4', '8.40188'], dtype='object')

In [39]:
cols = bad_info.columns
cols

Index(['7_2009', '3', '4', '8.40188'], dtype='object')

In [40]:
for col in cols:
    print(col)

7_2009
3
4
8.40188


In [41]:
bad_info

,7_2009,3,4,8.40188
count,1.950900e+04,19509.000000,19507.000000,19503.000000
mean,9.793011e+03,3.004910,9.549200,4.964280
std,8.202300e+04,0.352347,2.890984,2.886162
min,0.000000e+00,0.303240,0.650635,0.000469
25%,3.650000e+02,3.000000,7.000000,2.469750
50%,1.115000e+03,3.000000,9.000000,4.940950
75%,4.498000e+03,3.000000,11.000000,7.448870
max,8.391881e+06,35.000000,32.000000,9.999940


In [42]:
bad_df[bad_df.get_loc[0]]

AttributeError: 'DataFrame' object has no attribute 'get_loc'

In [43]:
col = bad_df['7_2009'][bad_df['7_2009'] != 0]
col

0         2930
1         4782
2          709
3          433
4        29861
         ...  
19504      310
19505     3236
19506     5054
19507     1550
19508      164
Name: 7_2009, Length: 19507, dtype: int64

In [44]:
len(col)

19507

In [46]:
import benford
freqs = benford.get_freqs(col)
freqs

{1: 5735,
 2: 3544,
 3: 2341,
 4: 1847,
 5: 1559,
 6: 1370,
 7: 1166,
 8: 1042,
 9: 903}

In [49]:
for x in freqs.keys():
    print(x)

1
2
3
4
5
6
7
8
9


In [52]:
actual = [freqs[x] for x in freqs.keys()]
actual

[5735, 3544, 2341, 1847, 1559, 1370, 1166, 1042, 903]

In [53]:
expected

NameError: name 'expected' is not defined

In [55]:
type(freqs.values())

dict_values

In [58]:
sort(freqs.keys())

NameError: name 'sort' is not defined